# Aggregated Sensitivity Analysis

In [ ]:
# Defaults
parameters_path = '../data/parameters/testbed.csv'
default_parameters = {}
book_paths = ['../runs/strong_scaling', '../runs/weak_scaling']

In [39]:
import scrapbook as sb
import numpy as np
from mhpc_project.utils import notebook_status, delta_mim
from mhpc_project.parameters import UniformSoilParameters as Parameters
from itertools import chain

In [ ]:
parameters = Parameters(parameters_path, default_parameters)
book = chain(sb.read_notebooks(where) for where in book_paths)

In [62]:
data = []
times = []
for book in book:
    for filename, nb in book.items():
        if notebook_status(nb) == 'completed': 
            log = nb.scraps['log'].data
            data.extend((x, l) for (_, x), l, _ in log if l != 'nan')


In [32]:
sa = delta_mim(parameters, data)

In [34]:
sa.sort_values('delta', key=np.abs, ascending=False)

,delta,delta_conf,S1,S1_conf
NVanGenuchten,0.251726,0.003981,0.417480,0.010034
RootDepth,0.167254,0.003334,0.212582,0.006860
ThetaSat,0.145829,0.003234,0.107025,0.005734
ThetaRes,0.135547,0.003290,0.119160,0.005671
AlphaVanGenuchten,0.089497,0.003548,0.042633,0.003162
WiltingPoint,0.086081,0.003392,0.039383,0.003199
LSAI,0.080011,0.002476,0.030497,0.003303
SoilRoughness,0.050154,0.002051,0.006045,0.001508
FieldCapacity,0.048582,0.002057,0.003846,0.001100
NormalHydrConductivity,0.046645,0.001714,0.003185,0.001079
